# Downloading sodapy API data

## Instructions

- Go to [Chicago Data Portal](https://data.cityofchicago.org/login) to sign in or create an account
- Generate an app token following [these steps](https://support.socrata.com/hc/en-us/articles/210138558-Generating-an-App-Token)
- Save the following environment variable

```
export CHI_API_KEY="my token"
```

- To access environment variable

```
from os import environ

environ["CHI_API_KEY"]
```

## Crashes data

In [2]:
import pandas as pd
import numpy as np
from sodapy import Socrata
pd.set_option('display.max_columns', None)

In [4]:
from os import environ

In [6]:
# environ["CHI_API_KEY"]

In [2]:
client = Socrata("data.cityofchicago.org", environ["CHI_API_KEY"])

In [3]:
results = client.get_all("85ca-t3if", content_type="json")

In [6]:
results_df = pd.DataFrame.from_records(results)

In [7]:
results_df.shape

(527633, 49)

In [21]:
results_df.head()

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,crash_type,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,location,intersection_related_i,hit_and_run_i,statements_taken_i,photos_taken_i,crash_date_est_i,private_property_i,dooring_i,work_zone_i,work_zone_type,workers_present_i,rd_no,lane_cnt
0,7a63945a9bae12c05dee82c1962bf40f23cdd02ca655d7...,2021-08-01T03:42:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",2021-08-01T03:42:00.000,DISREGARDING OTHER TRAFFIC SIGNS,UNABLE TO DETERMINE,4855,S,MORGAN ST,933,2,NO INDICATION OF INJURY,0,0,0,0,0,1,0,3,1,8,41.805384738,-87.650153056,"{'type': 'Point', 'coordinates': [-87.65015305...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,2021-08-01T02:41:00.000,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",2021-08-01T03:25:00.000,DISREGARDING TRAFFIC SIGNALS,NOT APPLICABLE,99,N,CENTRAL AVE,1522,2,NONINCAPACITATING INJURY,1,0,0,1,0,3,0,2,1,8,41.882148694,-87.764773775,"{'type': 'Point', 'coordinates': [-87.76477377...",Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bba4db7cfef29230aa5d24f6b8912a2539b5d6f481a94e...,2021-08-01T02:26:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",2021-08-01T02:26:00.000,IMPROPER TURNING/NO SIGNAL,IMPROPER TURNING/NO SIGNAL,1537,S,MICHIGAN AVE,132,2,NO INDICATION OF INJURY,0,0,0,0,0,4,0,2,1,8,41.860929115,-87.623859038,"{'type': 'Point', 'coordinates': [-87.62385903...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1e39110c6e01fd2f24e282cd610eb90d6f98a6241ab193...,2021-08-01T02:22:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",2021-08-01T02:25:00.000,UNABLE TO DETERMINE,UNABLE TO DETERMINE,5458,S,WELLS ST,935,2,NONINCAPACITATING INJURY,1,0,0,1,0,1,0,2,1,8,41.794560071,-87.631928475,"{'type': 'Point', 'coordinates': [-87.63192847...",N,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d20d9db8a88fc227d48392b45380ae57898e5fcdcda7db...,2021-08-01T02:08:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",2021-08-01T02:09:00.000,IMPROPER LANE USAGE,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,151,W,GRAND AVE,1831,3,NONINCAPACITATING INJURY,2,0,0,2,0,3,0,2,1,8,41.89148821,-87.633099826,"{'type': 'Point', 'coordinates': [-87.63309982...",NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
results_df = results_df.drop(columns=["location"])

In [68]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527633 entries, 0 to 527632
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   crash_record_id                527633 non-null  object
 1   crash_date                     527633 non-null  object
 2   posted_speed_limit             527633 non-null  object
 3   traffic_control_device         527633 non-null  object
 4   device_condition               527633 non-null  object
 5   weather_condition              527633 non-null  object
 6   lighting_condition             527633 non-null  object
 7   first_crash_type               527633 non-null  object
 8   trafficway_type                527633 non-null  object
 9   alignment                      527633 non-null  object
 10  roadway_surface_cond           527633 non-null  object
 11  road_defect                    527633 non-null  object
 12  report_type                    514502 non-nu

In [12]:
import psycopg2 as pg2
from sqlalchemy import create_engine
# from os import environ
from sqlalchemy.engine.base import Engine

In [13]:
def make_alchemy_engine(
        dbname: str='postgres', port: int=5432) -> Engine:
    username = environ['PG_USER']
    password = environ['PG_PASSWORD']
    host = environ['PG_HOST']
    string = f'postgresql://{username}:{password}@{host}:{port}/{dbname}'
    return create_engine(string)

In [71]:
alchemy_engine = make_alchemy_engine("chi-traffic-accidents")

In [72]:
results_df.to_sql(
    "crashes_raw", alchemy_engine, index=False, if_exists="replace")

In [73]:
alchemy_engine.dispose()
client.close()

## People data

In [8]:
soda_client = Socrata("data.cityofchicago.org", environ["CHI_API_KEY"])
results = soda_client.get_all("u6pd-qa9d", content_type="json")
results_df = pd.DataFrame.from_records(results)
results_df.shape

(1166012, 30)

In [9]:
results_df.head()

,person_id,person_type,crash_record_id,vehicle_id,crash_date,city,state,zipcode,sex,age,drivers_license_state,drivers_license_class,safety_equipment,airbag_deployed,ejection,injury_classification,driver_action,driver_vision,physical_condition,bac_result,seat_no,hospital,ems_agency,ems_run_no,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result_value,rd_no,cell_phone_use
0,O1152321,DRIVER,7a63945a9bae12c05dee82c1962bf40f23cdd02ca655d7...,1093058,2021-08-01T03:42:00.000,ORLAND PARK,IL,60462,M,39,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,OTHER,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O1152305,DRIVER,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,1093044,2021-08-01T02:41:00.000,NaN,NaN,NaN,M,NaN,NaN,NaN,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NONE,NO INDICATION OF INJURY,DISREGARDED CONTROL DEVICES,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O1152306,DRIVER,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,1093047,2021-08-01T02:41:00.000,CHICAGO,IL,60651,F,25,IL,NaN,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NONINCAPACITATING INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P258653,PASSENGER,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,1093047,2021-08-01T02:41:00.000,CHICAGO,IL,NaN,M,NaN,NaN,NaN,USAGE UNKNOWN,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P258654,PASSENGER,d207ce72f4348fa06dddcd98628d0dec81d4f195cb8d8b...,1093047,2021-08-01T02:41:00.000,BLUE ISLAND,IL,60406,F,NaN,NaN,NaN,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166012 entries, 0 to 1166011
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   person_id              1166012 non-null  object
 1   person_type            1166012 non-null  object
 2   crash_record_id        1166012 non-null  object
 3   vehicle_id             1143065 non-null  object
 4   crash_date             1166012 non-null  object
 5   city                   858745 non-null   object
 6   state                  868812 non-null   object
 7   zipcode                784098 non-null   object
 8   sex                    1148473 non-null  object
 9   age                    831949 non-null   object
 10  drivers_license_state  688648 non-null   object
 11  drivers_license_class  595129 non-null   object
 12  safety_equipment       1162597 non-null  object
 13  airbag_deployed        1144043 non-null  object
 14  ejection               1151806 non

In [14]:
alchemy_engine = make_alchemy_engine("chi-traffic-accidents")

In [15]:
results_df.to_sql(
    "people_raw", alchemy_engine, index=False, if_exists="replace")

In [16]:
alchemy_engine.dispose()
soda_client.close()